In [0]:
%run "./reader_factory"

In [0]:
%run "./Transformer_Notebook"

In [0]:
%run "./Extractor_Notebook"

In [0]:
%run "./Loader_Notebook"

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("The_Project").getOrCreate()

input_df = spark.read.format("csv").option("header" , True).load("dbfs:/FileStore/tables/Transaction_Updated.csv")

input_df.show()

+--------------+-----------+------------+----------------+
|transaction_id|customer_id|product_name|transaction_date|
+--------------+-----------+------------+----------------+
|            11|        105|      iPhone|      2022-02-01|
|            12|        106|      iPhone|      2022-02-02|
|            13|        107|     AirPods|      2022-02-03|
|            14|        105|     AirPods|      2022-02-04|
|            15|        108|      iPhone|      2022-02-05|
|            16|        106|     MacBook|      2022-02-06|
|            17|        107|      iPhone|      2022-02-07|
|            18|        105|     MacBook|      2022-02-08|
|            19|        108|     AirPods|      2022-02-09|
|            20|        106|     AirPods|      2022-02-10|
+--------------+-----------+------------+----------------+



In [0]:
class FirstWorkFlow:
    """
    etl pipeline to generate the data for all customers who have bought airpods just after buying iphone 
    """ 
    def __init__(self):
        pass
    def runner(self):
        
         # step 1 : Extract all required data from the different source 

        inputDfs =  AirpodsAfterIphoneExtractor().extract()

        # step 2 : Implement the transformation logic 
        
        firstTransformedDF = AirpodsAfterIphoneTransformer().transform(inputDfs)

        # step 3 : load all the required data to different sink 
        spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")

        AirPodsAfterIphoneLoader(firstTransformedDF).sink()






In [0]:
class SecondWorkFlow:
    """
    etl pipeline to generate the data for all customers who have bought  iphone and airpods 
    """ 
    def __init__(self):
        pass
    def runner(self):
        
         # step 1 : Extract all required data from the different source 

        inputDfs =  AirpodsAfterIphoneExtractor().extract()
         
        # step 2 : Implement the transformation logic 
        
        OnlyAirpodsAndIphoneDF = OnlyAirpodsAndIphone().transform(inputDfs)

        # step 3 : load all the required data to different sink 
        spark.conf.set("spark.databricks.delta.formatCheck.enabled", "false")

        OnlyAirpodsAndIphoneLoader(OnlyAirpodsAndIphoneDF).sink()

  

In [0]:
class WorkFlowRunner:
    def __init__(self,name):
        self.name = name 
    def runner(self):
        if self.name == "firstWorkFlow":
            return FirstWorkFlow().runner()
        elif self.name =="secondWorkFlow":
            return SecondWorkFlow().runner()
        else: 
            raise valueError (f"Not Implemented for {self.name} ")
name = "secondWorkFlow"  # default set to the secondWorkFlow
workFlowRunner =  WorkFlowRunner(name).runner()       


+--------------+-----------+------------+----------------+
|transaction_id|customer_id|product_name|transaction_date|
+--------------+-----------+------------+----------------+
|            11|        105|      iPhone|      2022-02-01|
|            14|        105|     AirPods|      2022-02-04|
|            18|        105|     MacBook|      2022-02-08|
|            12|        106|      iPhone|      2022-02-02|
|            16|        106|     MacBook|      2022-02-06|
|            20|        106|     AirPods|      2022-02-10|
|            13|        107|     AirPods|      2022-02-03|
|            17|        107|      iPhone|      2022-02-07|
|            15|        108|      iPhone|      2022-02-05|
|            19|        108|     AirPods|      2022-02-09|
+--------------+-----------+------------+----------------+

+-----------+-------------+---------+--------+
|customer_id|customer_name|join_date|location|
+-----------+-------------+---------+--------+
|        105|          Eva|     